In [24]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

2

# Imports

In [25]:
import asyncio
from nse import NSEfnos, equity_iv_df, RBI
from utils import append_safe_strikes, append_black_scholes, append_cos, load_config, merge_and_overwrite_df, append_xPrice
from ibfuncs import marginsAsync

# Inputs

In [27]:
symbol = 'PNB'
config = load_config()
port = config.get("PORT")

In [28]:
def make_early_opts_for_symbol(
    symbol: str,  # nse_symbol. can be equity or index
    port: int,
    timeout: int=2, # timeout for marginsAsync
        ) -> pd.DataFrame:
    """Make target options for nakeds with earliest expiry
    
    Args:
       symbol: can be equity or index
       port: int. Could be LIVE_PORT | PAPER_PORT
       """

    # initialize and get the base df
    n = NSEfnos()
    q = n.stock_quote_fno(symbol)
    dfe = equity_iv_df(q)
    
    # clean up zero IVs and dtes
    mask = (dfe.iv > 0) & (dfe.dte > 0)
    df = dfe[mask].reset_index(drop=True)
    
    # Append safe strikes
    df = append_safe_strikes(df)
    
    # Append black scholes
    
    rbi = RBI()
    risk_free_rate = rbi.repo_rate() / 100
    
    df = append_black_scholes(df, risk_free_rate)
    
    # Append contract, order to prep for margin
    df = append_cos(df)
    
    # Get margins with approrpriate timeout and append
    df_mcom = asyncio.run(marginsAsync(df, port, timeout=timeout))
    
    df = merge_and_overwrite_df(df, df_mcom)
    
    # Append xPrice
    df = append_xPrice(df)

    return df

In [29]:
make_early_opts_for_symbol(symbol, port)

,nse_symbol,ib_symbol,instrument,expiry,undPrice,safe_strike,right,strike,dte,hv,iv,lot,price,sdev,intrinsic,bsPrice,contract,comm,margin,xPrice,rom
4,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,142,C,145.0,28.98,0.48,0.36,8000,0.45,12.70,0.0,0.42,"Option(conId=707119770, symbol='PNB', lastTrad...",20.0,1341.41,0.45,33.81
2,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,141,C,140.0,28.98,0.48,0.34,8000,0.75,11.98,1.0,0.70,"Option(conId=707119760, symbol='PNB', lastTrad...",20.0,31529.50,1.75,5.59
9,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,138,C,120.0,28.98,0.48,0.28,8000,6.70,9.65,18.0,6.48,"Option(conId=707119729, symbol='PNB', lastTrad...",20.0,131719.99,24.70,18.90
1,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,138,C,125.0,28.98,0.48,0.29,8000,4.00,10.05,13.0,3.80,"Option(conId=707119739, symbol='PNB', lastTrad...",20.0,114706.96,17.00,14.94
6,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,139,C,127.5,28.98,0.48,0.29,8000,3.00,10.24,11.5,2.84,"Option(conId=712933032, symbol='PNB', lastTrad...",20.0,103512.56,14.50,14.12
0,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,139,C,130.0,28.98,0.48,0.31,8000,2.30,10.67,9.0,2.17,"Option(conId=707119745, symbol='PNB', lastTrad...",20.0,91012.91,11.30,12.51
8,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,107,P,125.0,28.98,0.48,0.31,8000,4.40,11.00,18.0,4.61,"Option(conId=707119742, symbol='PNB', lastTrad...",20.0,116632.93,22.60,19.53
3,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,140,C,135.0,28.98,0.48,0.31,8000,1.20,10.94,5.0,1.12,"Option(conId=707119755, symbol='PNB', lastTrad...",20.0,62336.14,6.20,10.02
5,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,107,P,120.0,28.98,0.48,0.31,8000,2.20,10.78,13.0,2.33,"Option(conId=707119732, symbol='PNB', lastTrad...",20.0,95508.98,15.35,16.20
7,PNB,PNB,STKOPT,2024-08-29 10:00:00+00:00,123.93,107,P,115.0,28.98,0.48,0.32,8000,1.05,11.24,8.0,1.12,"Option(conId=707119727, symbol='PNB', lastTrad...",20.0,66495.50,9.10,13.79
